In [5]:
from math import *
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon, Circle, Wedge
from matplotlib import animation
import matplotlib as mpl
import numpy as np
from matplotlib.widgets import Slider, Button, RadioButtons
import ipywidgets as widgets

global p0, a0,baseline,d1,d2,wavelength,maxint


a0=[1]




def create_figure(pos,baseline,d1,d2,wavelength):
    #baseline = separation between the slits
    baseline /= 2. #baseline = baseline/2.
    maxint=None
    
    ax = plt.axes(frameon=False)
    
    #disable outer frame
    plt.xlim(-d1-.1, d2+2) and plt.ylim(-1, 1)#limit the frame size
    plt.xticks([]) and plt.yticks([])
    plt.axhline(0, ls=':')

    

    ## draw representation of slits
    ax.arrow(0, 1,0, baseline-1, lw=0, width=.1, head_width=.1, length_includes_head=True)
    ax.arrow(0,-1,0, 1-baseline, lw=0, width=.1, head_width=.1, length_includes_head=True)
    ax.arrow(0, 0,0,  baseline,  lw=0, width=.1, head_width=.1, length_includes_head=True)
    ax.arrow(0, 0,0, -baseline,  lw=0, width=.1, head_width=.1, length_includes_head=True)
    ## draw representation of lightpath from slits to centre of screen
    ax.arrow(0, baseline,d2,-baseline, length_includes_head=True)
    ax.arrow(0,-baseline,d2, baseline, length_includes_head=True)
    p0=[pos]
    ## draw representation of sinewave from the central position
    xw = np.arange(-d1, -d1+(d1+d2)/4, .01)
    yw = np.sin(2*np.pi*xw/wavelength)*.1 + (p0[0]+p0[-1])/2
    l1,=ax.plot(xw,yw,'b')
    ## 'xs' is a vector of x cordinates on the screen
    ## and we accumulate the interference pattern for each source into 'pattern'
    xs = np.arange(-1, 1, .01) 
    pattern = 0
    total_intensity = 0
    ## compute contribution to pattern from each source position p
    for p,a in np.broadcast(p0,a0):
        s,=ax.plot(-d1, p, marker='o', ms=10, mfc='red', mew=0)
        total_intensity += a
        if p == p0[0] or p == p0[-1]:
            p1=ax.arrow(-d1, p, d1, baseline-p, length_includes_head=True)
            p2=ax.arrow(-d1, p, d1,-baseline-p, length_includes_head=True)
        # compute the two pathlenghts
        path1 = np.sqrt(d1**2 + (p-baseline)**2) + np.sqrt(d2**2 + (xs-baseline)**2)
        path2 = np.sqrt(d1**2 + (p+baseline)**2) + np.sqrt(d2**2 + (xs+baseline)**2)
        diff = path1 - path2
    # calculate interference pattern from this source
    pattern = pattern + a*np.cos(2*np.pi*diff/wavelength) 
    maxint = maxint or total_intensity
    # add fake axis to interference pattern just to make it a "wide" image
    pattern_image = pattern[:,np.newaxis] + np.zeros(10)[np.newaxis,:]
    pat=ax.imshow(pattern_image, extent=(d2,d2+1,-1,1), cmap=plt.gray(), vmin=-maxint, vmax=maxint)
    # make a plot of the interference pattern
    patc,=ax.plot(d2+1.5+pattern/(maxint*2), xs, 'r')


style = {'description_width': 'initial'}    #to be abto to show full description of the sliders
@widgets.interact(position=widgets.FloatSlider(
    value=0,
    min=-1, 
    max=1, 
    step=0.001, 
    description='Position',style=style
    ),baseline=widgets.FloatSlider(
    value=1,
    min=0, 
    max=2, 
    step=0.001, 
    description='Separation b/w slits',style=style
    ),d1=widgets.FloatSlider(
    value=5,
    min=1, 
    max=5, 
    step=0.001, 
    description='d1',style=style
    ),d2=widgets.FloatSlider(
    value=5,
    min=1, 
    max=5, 
    step=0.001, 
    description='d2',style=style
    ),wavelength=widgets.FloatSlider(
    value=0.1,
    min=0.01, 
    max=0.5, 
    step=0.01, 
    description='Wavelength',style=style
    ))
def update(position,baseline,d1,d2,wavelength):
    plt.figure(figsize=(20, 20))
    ax=plt.gca()
    plt.cla()
    create_figure(position,baseline,d1,d2,wavelength)

interactive(children=(FloatSlider(value=0.0, description='Position', max=1.0, min=-1.0, step=0.001, style=Slid…